In [1]:
import requests
import json
import random
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

PATH_TO_IMG = "IMG_20201210_160628.jpg"
PATH_TO_MODEL = "../models/RV2-NASNetMobile-non-trainable"


In [2]:
# load model
model = load_model(PATH_TO_MODEL)


In [3]:
# load image and process
def get_proc(path):
    width = 75
    height = 150
    
    img = Image.open(path)
    img = img.resize((width, height), resample=1)
    img = img_to_array(img)
    img = img/255
    img = img.astype('float32')
    img = np.expand_dims(img, axis=0)
    
    return img
    
# prediction
def get_pred(img):
    return model.predict(img)[0,0]


# post_process
def post_process(pred):
    labels = ['good', 'kyphosis']
    threshold = 0.5
    
    if pred >= threshold:
        index = 1
        
    else:
        index = 0
        
    return labels[index]


def pipe(path):
    img = get_proc(path)
    pred = get_pred(img)
    label = post_process(pred)
    
    return label

In [90]:
APPARATUS = "Bodyweight"
PART_OF_BODY = "Shoulders"


def get_token(username,password):
    response = requests.post('http://204.235.60.194/consumer/login', data={'username':username, 'password':password})
    token =response.json()
    return token


def request(param, token, n_exercise):
    url = 'http://204.235.60.194/exrxapi/v1/allinclusive/exercises?'
    keys_to_extract = ['Apparatus_Name','Exercise_Name_Complete_Abbreviation','Small_Img_1','Small_Img_2','Larg_Img_1',\
                       'Instructions_Preparation','Instructions_Execution','video_src','GIF_Img']
    
    response = requests.get(url, params=param, headers={'Authorization': "Bearer "+token['token']})
    response = json.loads(response.text)['exercises']
    
    exercices = np.random.choice(response,n_exercise,replace=False)
    ex_list = [dict((key, value) for (key, value) in exercice.items() if key in keys_to_extract) for exercice in exercices]

    return ex_list
        
    
def main(apparatus, part_of_body):
    token = get_token('raneenaexrx','StKhg6Zu')
    stretch_params = {'stretch':'', 'bodypart':[part_of_body]}
    strength_params = {'bodypart':[part_of_body], 'apparatus':[apparatus]}
    
    response_stretch = request(stretch_params, token,1)
    response_strength = request(strength_params, token,2)
    
    return response_stretch, response_strength
    
    
def activate():
    if pipe(PATH_TO_IMG) == 'kyphosis':
        return main(APPARATUS, PART_OF_BODY)

In [91]:
stretch, strength = activate()


In [92]:
stretch

[{'Apparatus_Name': 'Stretch',
  'Instructions_Preparation': 'Face stationary bar. Grasp stationary bar with one hand approximately face height. Rotate body so upper arm is positioned across neck.',
  'Instructions_Execution': 'Turn body into upper arm. Hold stretch. Repeat with opposite arm.',
  'Small_Img_1': '//app.exrx.net/upload/images/5b94e4d3046e7.jpg',
  'Small_Img_2': '//app.exrx.net/',
  'Larg_Img_1': '//app.exrx.net/upload/images/5b94e4dacefd2.jpg',
  'GIF_Img': '//app.exrx.net/',
  'video_src': '',
  'Exercise_Name_Complete_Abbreviation': 'Fixed Bar Rear Deltoid Stretch'}]

In [93]:
strength

[{'Apparatus_Name': 'Bodyweight',
  'Instructions_Preparation': 'Lay on back with belly under fixed horizontal bar. Bend knees and position feet on floor. Grasp bar with wide overhand grip. Turn elbows outward to sides.\nWalk forward under bar while pulling upper chest close to bar. With heels on floor, position body at angle under bar with legs, hips and spine straight. Position forearms perpendicular to body, orientated forward. Also, position upper arms perpendicular to body, orientated to each side.',
  'Instructions_Execution': 'Keeping hips on floor, low back straight, and elbow pointed out to sides; pull torso up to bar. Return when upper arms are just beyond parallel to one another. Lower body downward until arms are extended straight. Return and repeat.',
  'Small_Img_1': '//app.exrx.net/upload/images/5b647c46210f5.jpg',
  'Small_Img_2': '//app.exrx.net/upload/images/5b647c4ae85f4.jpg',
  'Larg_Img_1': '//app.exrx.net/upload/images/5b647c5572d15.jpg',
  'GIF_Img': '//app.exrx.